In [15]:
import cloudscraper
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd  
import time 
import random

In [28]:
def cook_soup(url):
    user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"]
    session = requests.session()
    scraper = cloudscraper.create_scraper(sess=session,delay=6)
    response = scraper.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    soup = BeautifulSoup(response.text,'lxml')
    print(f'url: {url} - status:{response.status_code}')
    return soup 





In [21]:
wild_sour_beer=["/beer/styles/87/",
"/beer/styles/198/" , 
"/beer/styles/15/" ,
"/beer/styles/52/" ,
"/beer/styles/53/" ,
"/beer/styles/10/" ,
"/beer/styles/234/" ,
"/beer/styles/16/" ,
"/beer/styles/14/" ,
"/beer/styles/50/" ,
"/beer/styles/171/"
]

#a list of href for wild sour beer (each link has a list to itterate through)


In [27]:

beerlist=[]
root='https://www.beeradvocate.com'
for element in wild_sour_beer:
    web_append=f'{root}{element}'
    for x in range(3500,100000,50):
        webber=f'{web_append}?sort=revsD&start={x}'
        soup=cook_soup(webber)
        time.sleep(10)
        table=soup.find('table')
        beerlinks=[root + link['href'] for link in table.find_all("a", href=re.compile("profile"))][::2]
        print(beerlinks)
        if len(beerlinks) == 0:
            break
        else:        
            for link in beerlinks:
                soup=cook_soup(link)
                name=soup.h1.find(string=True)
                beerstats=soup.find('dl',{'class': 'beerstats'})
                try:    
                    brewery=beerstats.find_all('dd',{'class':'beerstats'})[0].text
                except:
                    brewery= 'Na'
                try:
                    location=beerstats.find_all('dd',{'class':'beerstats'})[1].text
                except:
                    location= 'Na'
                try:
                    type_rank=beerstats.find_all('dd',{'class':'beerstats'})[2].text
                except:
                    type_rank= 'Na'
                try:
                    type_=beerstats.find_all('a',{'class':'Tooltip'})[1].text
                except:
                    type_= 'Na'
                try:
                    rank=beerstats.find_all('a',{'class':'Tooltip'})[2].text
                except:
                    rank= 'Na'
                try:
                    abv=beerstats.find_all('dd',{'class':'beerstats'})[3].text
                except:
                    abv= 'Na'
                try:
                    score_orank=beerstats.find_all('dd',{'class':'beerstats'})[4].text
                except:
                    score_orank= 'Na'
                try:
                    score=beerstats.find('span',{'class':'ba-score Tooltip'}).text
                except:
                    score= 'Na'
                try:
                    overall_rank=beerstats.find_all('a',{'class':'Tooltip'})[3].text
                except:
                    overall_rank= 'Na'
                try:
                    avg_and_pdev=beerstats.find_all('dd',{'class':'beerstats'})[5].text
                except:
                    avg_and_pdev= 'Na'
                try:
                    avg_rating=beerstats.find('span',{'class':'ba-ravg Tooltip'}).text
                except:
                    avg_rating= 'Na'
                try:
                    pdev=beerstats.find('span',{'class':'ba-pdev muted Tooltip'}).text
                except:
                    pdev= 'Na'
                try:
                    reviews=beerstats.find_all('dd',{'class':'beerstats'})[6].text
                except:
                    reviews= 'Na'
                try:
                    ratings=beerstats.find_all('dd',{'class':'beerstats'})[7].text
                except:
                    ratings= 'Na'
                try:
                    status=beerstats.find_all('dd',{'class':'beerstats'})[8].text
                except:
                    status= 'Na'
                try:
                    date_added=beerstats.find_all('dd',{'class':'beerstats'})[10].text
                except:
                    date_added= 'Na'
                beer_info={
                            'name': name,
                            'brewery': brewery,
                            'location':location,
                            'type_rank':type_rank,
                            'type_': type_,
                            'rank':rank,
                            'abv': abv,
                            'score_orank': score_orank,
                            'score': score,
                            'overall_rank':overall_rank,
                            'avg_and_pdev': avg_and_pdev,
                            'avg_rating': avg_rating,
                            'pdev':pdev,
                            'reviews': reviews,
                            'ratings': ratings,
                            'status':status,
                            'date_added':date_added
                                }

                beerlist.append(beer_info)
                print(f'{x}-{link}-{name}')


ConnectionError: HTTPSConnectionPool(host='www.beeradvocate.com', port=443): Max retries exceeded with url: /beer/styles/87/?sort=revsD&start=3500 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x275bec950>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [26]:
df=pd.DataFrame(beerlist)
df.to_csv('../data/wild_sour_3500_1.csv')